In [11]:
# F1 Data Engineering Project

StatementMeta(, 11f6a06c-ce7e-4de7-98e0-e53d5fb84e66, 15, Finished, Available, Finished)

In [12]:
race_year = 2024
race_name = 'Silverstone'

StatementMeta(, 11f6a06c-ce7e-4de7-98e0-e53d5fb84e66, 16, Finished, Available, Finished)

In [13]:
#%pip install fastf1

StatementMeta(, 11f6a06c-ce7e-4de7-98e0-e53d5fb84e66, 17, Finished, Available, Finished)

In [14]:
import fastf1

# Get the schedule
schedule = fastf1.get_event_schedule(2025)

#Show the data
display(schedule)

StatementMeta(, 11f6a06c-ce7e-4de7-98e0-e53d5fb84e66, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6a09bedf-9566-4c92-b08e-d248e7d67a27)

In [15]:
import os

# Check if the folder exists
path = "/lakehouse/default/Files"

if os.path.exists(path):
    print("✅ Connection Successful! The folder exists.")
else:
    print("❌ Still not found. Please double-check the Lakehouse is added on the left.")

StatementMeta(, 11f6a06c-ce7e-4de7-98e0-e53d5fb84e66, 19, Finished, Available, Finished)

✅ Connection Successful! The folder exists.


In [16]:
file_path = "//lakehouse/default/Files/2025_race_schedule.csv"

schedule.to_csv(file_path,index=False)

print(f"Success! File saved to: {file_path}")

StatementMeta(, 11f6a06c-ce7e-4de7-98e0-e53d5fb84e66, 20, Finished, Available, Finished)

Success! File saved to: //lakehouse/default/Files/2025_race_schedule.csv


In [17]:
%%sql
SELECT EventName, EventDate, Location
FROM race_schedule_2025
WHERE Country = 'Monaco'

StatementMeta(, 11f6a06c-ce7e-4de7-98e0-e53d5fb84e66, 21, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 3 fields>

In [18]:
# 1. Select the specific race (Monaco 2024, Race Session)
session = fastf1.get_session(race_year, race_name, 'R')

# 2. Download the data (The heavy lifting)
print("Downloading race data... this may take a minute...")
session.load()

# 3. Extract the 'Laps' data
monaco_laps = session.laps

# 4. Filter for just the columns we care about for our App
# (Driver, LapNumber, LapTime, Tire, Compound)
clean_laps = monaco_laps[['Driver', 'LapNumber', 'LapTime', 'TyreLife', 'Compound']].copy()

# We add the race name to every single row so we know where this data came from
clean_laps['RaceName'] = race_name
clean_laps['Season'] = race_year

# 5. Convert to String for easier storage (removes complex time objects)
clean_laps['LapTime'] = clean_laps['LapTime'].astype(str)

# 6. Show the first few rows
display(clean_laps.head())

StatementMeta(, 11f6a06c-ce7e-4de7-98e0-e53d5fb84e66, 22, Finished, Available, Finished)

req            INFO 	Using cached data for car_data
req            INFO 	No cached data found for position_data. Loading data...
_api           INFO 	Fetching position data...
_api           INFO 	Parsing position data...
_api        WARNING 	Driver 21: Position data is incomplete!
_api        WARNING 	Driver  3: Position data is incomplete!
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for weather_data. Loading data...
_api           INFO 	Fetching weather data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for race_control_messages. Loading data...
_api           INFO 	Fetching race control messages...
req            INFO 	Data has been written to cache!
core           INFO 	Finished loading data for 20 drivers: ['44', '1', '4', '81', '55', '27', '18', '14', '23', '22', '2', '20', '3', '16', '77', '31', '11', '24', '63', '10']


SynapseWidget(Synapse.DataFrame, d80f0823-c396-47c7-8691-d59ea51a44fe)

In [19]:
# Convert to Spark
spark_laps = spark.createDataFrame(clean_laps)

# Save as a new Delta Table
spark_laps.write.mode("append").saveAsTable("f1_race_laps")

print("✅ Lap data saved to table 'f1_race_laps'")

StatementMeta(, 11f6a06c-ce7e-4de7-98e0-e53d5fb84e66, 23, Finished, Available, Finished)

✅ Lap data saved to table 'f1_race_laps'


In [20]:
%%sql
SELECT 
    Driver, 
    MIN(LapTime) as Fastest_Lap_Time,
    COUNT(*) as Total_Laps_Completed
FROM monaco_2024_laps
GROUP BY Driver
ORDER BY Fastest_Lap_Time ASC

StatementMeta(, 11f6a06c-ce7e-4de7-98e0-e53d5fb84e66, 24, Finished, Available, Finished)

<Spark SQL result set with 20 rows and 3 fields>

In [23]:
# 1. Load the Silver Table (All Laps)
df_laps = spark.read.table("f1_race_laps")

# 2. Register as a View so we can use SQL
df_laps.createOrReplaceTempView("laps_view")

# 3. Create the Gold Data (The Aggregation)
gold_stats = spark.sql("""
    SELECT 
        RaceName,
        Season,
        Driver,
        MIN(LapTime) as Fastest_Lap,
        COUNT(*) as Laps_Completed,
        AVG(TyreLife) as Avg_Tyre_Life
    FROM laps_view
    GROUP BY RaceName, Season, Driver
    ORDER BY RaceName, Fastest_Lap ASC
""")

# 4. Save the Table (This is the critical step!)
gold_stats.write.mode("overwrite").saveAsTable("gold_driver_stats")

print("✅ Gold Table 'gold_driver_stats' created successfully!")

StatementMeta(, 11f6a06c-ce7e-4de7-98e0-e53d5fb84e66, 27, Finished, Available, Finished)

✅ Gold Table 'gold_driver_stats' created successfully!


In [24]:
%%sql
SELECT * FROM gold_driver_stats LIMIT 5

StatementMeta(, 11f6a06c-ce7e-4de7-98e0-e53d5fb84e66, 28, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 6 fields>

In [25]:
%%sql
SELECT RaceName, Season, COUNT(*) as Total_Laps
FROM f1_race_laps
GROUP BY RaceName, Season

StatementMeta(, 11f6a06c-ce7e-4de7-98e0-e53d5fb84e66, 29, Finished, Available, Finished)

<Spark SQL result set with 4 rows and 3 fields>